In [1]:
# import the required libraries

import pandas as pd
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras import layers, optimizers
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import os
import random

In [2]:
# read the file containing MRI image path and labels
brain_df = pd.read_csv('Brain_MRI/data_mask.csv')

In [3]:
# 0 - No Tumor (2556 Images)
# 1 - Tumor (1373 Images)
brain_df['mask'].value_counts()

0    2556
1    1373
Name: mask, dtype: int64

In [4]:
# Drop the patient id column
brain_df_train = brain_df.drop(columns = ['patient_id'])
brain_df_train.shape

(3929, 3)

In [5]:
# Convert the data in mask column to string format, to use categorical mode in flow_from_dataframe
brain_df_train['mask'] = brain_df_train['mask'].apply(lambda x: str(x))

In [7]:
brain_df_train.head()

,image_path,mask_path,mask
0,TCGA_CS_5395_19981004/TCGA_CS_5395_19981004_1.tif,TCGA_CS_5395_19981004/TCGA_CS_5395_19981004_1_...,0
1,TCGA_CS_4944_20010208/TCGA_CS_4944_20010208_1.tif,TCGA_CS_4944_20010208/TCGA_CS_4944_20010208_1_...,0
2,TCGA_CS_4941_19960909/TCGA_CS_4941_19960909_1.tif,TCGA_CS_4941_19960909/TCGA_CS_4941_19960909_1_...,0
3,TCGA_CS_4943_20000902/TCGA_CS_4943_20000902_1.tif,TCGA_CS_4943_20000902/TCGA_CS_4943_20000902_1_...,0
4,TCGA_CS_5396_20010302/TCGA_CS_5396_20010302_1.tif,TCGA_CS_5396_20010302/TCGA_CS_5396_20010302_1_...,0


In [8]:
# split the data into train and test data

from sklearn.model_selection import train_test_split

train, test = train_test_split(brain_df_train, test_size = 0.15)

In [9]:
# create a image generator
from keras_preprocessing.image import ImageDataGenerator

# Create a data generator which scales the data from 0 to 1 and makes validation split of 0.15
datagen = ImageDataGenerator(rescale=1./255., validation_split = 0.15)

In [10]:
train_generator=datagen.flow_from_dataframe(dataframe=train,
                                            directory= './Brain_MRI/',
                                            x_col='image_path',
                                            y_col='mask',
                                            subset="training",
                                            batch_size=16,
                                            shuffle=True,
                                            class_mode="categorical",
                                            target_size=(128,128))


valid_generator=datagen.flow_from_dataframe(dataframe=train,
                                            directory= './Brain_MRI/',
                                            x_col='image_path',
                                            y_col='mask',
                                            subset="validation",
                                            batch_size=16,
                                            shuffle=True,
                                            class_mode="categorical",
                                            target_size=(128,128))

Found 2839 validated image filenames belonging to 2 classes.
Found 500 validated image filenames belonging to 2 classes.


In [11]:
# Create a data generator for test images
test_datagen=ImageDataGenerator(rescale=1./255.)

test_generator=test_datagen.flow_from_dataframe(dataframe=test,
                                                directory= './Brain_MRI/',
                                                x_col='image_path',
                                                y_col='mask',
                                                batch_size=16,
                                                class_mode='categorical',
                                                target_size=(128,128))

Found 590 validated image filenames belonging to 2 classes.


In [12]:
# Load pretrained model (best saved one)
with open('./Saved Model/classifier_model.json', 'r') as json_file:
    json_savedModel= json_file.read()
    
# load the model  
model = tf.keras.models.model_from_json(json_savedModel)
model.load_weights('./Saved Model/classifier_weights.h5')
model.compile(loss = 'sparse_categorical_crossentropy', optimizer=optimizers.Adam(learning_rate=0.0001), metrics= ["accuracy"])

## Quantization Aware Training

In [16]:
import tensorflow_model_optimization as tfmot

quantize_model = tfmot.quantization.keras.quantize_model

# q_aware stands for for quantization aware.
q_aware_model = quantize_model(model)

# `quantize_model` requires a recompile.
q_aware_model.compile(optimizer=optimizers.Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

q_aware_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
quantize_layer_1 (QuantizeLa (None, 128, 128, 3)       3         
_________________________________________________________________
quant_conv2d (QuantizeWrappe (None, 128, 128, 32)      963       
_________________________________________________________________
quant_max_pooling2d (Quantiz (None, 64, 64, 32)        1         
_________________________________________________________________
quant_conv2d_1 (QuantizeWrap (None, 64, 64, 64)        18627     
_________________________________________________________________
quant_max_pooling2d_1 (Quant (None, 32, 32, 64)        1         
_________________________________________________________________
quant_dropout (QuantizeWrapp (None, 32, 32, 64)        1         
_________________________________________________________________
quant_conv2d_2 (QuantizeWrap (None, 32, 32, 64)        3

In [17]:
history = q_aware_model.fit(train_generator, validation_data= valid_generator, epochs = 5)

Epoch 1/5
178/178 [==============================] - 33s 155ms/step - loss: 0.1012 - accuracy: 0.9718 - val_loss: 0.0280 - val_accuracy: 0.9940
Epoch 2/5
178/178 [==============================] - 25s 139ms/step - loss: 0.0900 - accuracy: 0.9711 - val_loss: 0.0282 - val_accuracy: 0.9960
Epoch 3/5
178/178 [==============================] - 25s 139ms/step - loss: 0.0736 - accuracy: 0.9768 - val_loss: 0.0279 - val_accuracy: 0.9920
Epoch 4/5
178/178 [==============================] - 25s 138ms/step - loss: 0.0681 - accuracy: 0.9785 - val_loss: 0.0257 - val_accuracy: 0.9920
Epoch 5/5
178/178 [==============================] - 24s 137ms/step - loss: 0.0685 - accuracy: 0.9775 - val_loss: 0.0251 - val_accuracy: 0.9920


In [18]:
scores = q_aware_model.evaluate(test_generator)

37/37 [==============================] - 3s 85ms/step - loss: 0.0403 - accuracy: 0.9864


In [19]:
converter = tf.lite.TFLiteConverter.from_keras_model(q_aware_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]

tflite_qaware_model = converter.convert()

INFO:tensorflow:Assets written to: C:\Users\kanak\AppData\Local\Temp\tmpnhol931o\assets


INFO:tensorflow:Assets written to: C:\Users\kanak\AppData\Local\Temp\tmpnhol931o\assets


In [22]:
print("Model size reduced from 2.29 MB to", np.around(len(tflite_qaware_model)/(1024),2), "KB")

Model size reduced from 2.29 MB to 196.98 KB


In [23]:
with open("brain_tumor_classifier.tflite", 'wb') as f:
    f.write(tflite_qaware_model)